Checking ingletons inequality for the W state on 6 qubits

In [1]:
from QIP import *
n=6
W_state_dict = {}
for i in range(n):
    bitstring = "0"*n
    bitstring = bitstring[:i] + '1' + bitstring[i+1:]
    W_state_dict[bitstring] = 1/n**.5
W_state = QCircuit(6,initial_state=bitstrings_to_vector(W_state_dict))
W_state.subsysem_analysis()
print("W state: "+state_as_string(W_state_dict))
# prints an empty list if ingletons never fails
print("Average failure saturation of ingletons inequality: "+str(W_state.ing["avg_fail_saturation"]))

W state: 0.408|100000> + 0.408|010000> + 0.408|001000> + 0.408|000100> + 0.408|000010> + 0.408|000001>
Average failure saturation of ingletons inequality: []


Checking Ingleton on the state $\ket{\psi}=\frac{1}{4}(\ket{0000}+\ket{1111}+\ket{1010}+\ket{1001})$

In [2]:
state_dict = {}
state_dict['0000']=1/4
state_dict['1111']=1/4
state_dict['1010']=1/4
state_dict['1001']=1/4
ing_test = QCircuit(4,initial_state=bitstrings_to_vector(state_dict))
ing_test.subsysem_analysis()
print("Average failure saturation of ingletons inequality: "+str(ing_test.ing["avg_fail_saturation"]))

Average failure saturation of ingletons inequality: []


Checking Ingleton using $\rho=\frac{1}{2}\ket{\psi}\bra{\psi}+\frac{1}{4}\ket{1010}\bra{1010}+\frac{1}{4}\ket{1001}\bra{1001}$, where $\ket{\psi}=\frac{1}{\sqrt{2}}(\ket{0000}+\bra{1111})$

In [3]:
psi_dict = {}
psi_dict["0000"]=1/2**.5
psi_dict["1111"]=1/2**.5
psi_vec = bitstrings_to_vector(psi_dict)
psi_vec.shape=(2**4,1)
vec1 = bitstrings_to_vector("1010")
vec1.shape=(2**4,1)
vec2 = bitstrings_to_vector("1001")
vec2.shape=(2**4,1)
rho = 0.5*np.outer(psi_vec,psi_vec.conj().T)+0.25*np.outer(vec1,vec1.conj().T)+0.25*np.outer(vec2,vec2.conj().T)

ing_test = QCircuit(4)
ing_test.density_matrix=rho
ing_test.subsysem_analysis()
print("Average failure saturation of ingletons inequality: "+str(ing_test.ing["avg_fail_saturation"]))

Average failure saturation of ingletons inequality: []
